In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
print(torch.__version__)

import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

batch_size = 32

train_dataset = datasets.MNIST('./data',
                               train=True,
                               download=True,
                               transform=transforms.ToTensor())

validation_dataset = datasets.MNIST('./data',
                                    train=False,
                                    transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                                batch_size=batch_size,
                                                shuffle=False)

for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)

model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

print(model)

2.0.1+cu118
Using PyTorch version: 2.0.1+cu118  Device: cpu
X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
Net(
  (fc1): Linear(in_features=784, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=10, bias=True)
)


In [2]:
def train(epoch, log_interval=200):
    # Set model to training mode
    model.train()

    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad()

        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()

        # Update weights
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)

    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))

In [3]:
epochs = 5

lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    validate(lossv, accv)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298854
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.977734
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.244394
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.791230
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.707093
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.478044
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.414540
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.250297
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.384215
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.551589

Validation set: Average loss: 0.3600, Accuracy: 8967/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.448805
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.270290
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.204037
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.347269
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.353995
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.139851
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.056438
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.099213
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.30